## 켈리 방정식(Kelly Criterion)
켈리 방정식은 장기적으로 수익을 기하급수적으로 극대화하기 위해 각 베팅 또는 투자에 자본의 얼마를 할당해야 하는지를 계산하는 공식입니다. 1956년 벨 연구소의 과학자였던 존 켈리(John L. Kelly Jr.)에 의해 개발되었으며, 처음에는 통신 이론의 노이즈 문제를 해결하기 위해 고안되었지만, 곧바로 전설적인 투자자들과 전문 도박사들에게 채택되어 자금 관리의 핵심 원칙으로 자리 잡았습니다.

핵심 아이디어는 ```"수익률의 로그 값을 최적화하여 장기적인 자산 성장을 극대화하는 것"```입니다. 단순히 한 번의 큰 수익을 노리는 것이 아니라, 수많은 시행을 거쳤을 때 파산의 위험을 최소화하면서 가장 높은 복리 수익을 달성하는 최적의 베팅 비율을 알려줍니다.

In [1]:
from scipy.stats import norm

# 표준편차의 배수 설정
sigma = 4

# 양쪽 꼬리(tail)의 확률을 구해야 하므로, 한쪽 꼬리의 확률 P(Z < -4)를 구해 2를 곱합니다.
prob_outside = norm.cdf(-sigma)
prob_outside

3.167124183311986e-05

In [2]:
print(f"백분율로 변환: {prob_outside:.8f}%")

백분율로 변환: 0.00003167%


In [3]:
# DPMO (Defects Per Million Opportunities) 계산
dpmo = prob_outside * 100000
print(f"십만 개 중 발생 건수 (DPMO): 약 {dpmo:.4f} 건")

십만 개 중 발생 건수 (DPMO): 약 3.1671 건


In [4]:
def calculate_kelly_fraction(p: float, b: float) -> float:
    """
    켈리 방정식을 사용하여 최적의 베팅 비율을 계산합니다.

    Args:
        p (float): 승리 확률 (0과 1 사이)
        b (float): 배당률 (손익비, 예: 1000원 걸어 2000원 순수익 시 b=2)

    Returns:
        float: 최적의 베팅 비율 f*. 0보다 작으면 0을 반환.
    """
    if not (0 <= p <= 1):
        raise ValueError("승리 확률(p)은 0과 1 사이의 값이어야 합니다.")
    if b <= 0:
        raise ValueError("배당률(b)은 0보다 커야 합니다.")
        
    q = 1 - p
    f_star = (b * p - q) / b
    
    return max(0, f_star)

# 위에서 들었던 예시로 계산해보기
p_coin = 0.3
b_coin = 3
kelly_fraction = calculate_kelly_fraction(p_coin, b_coin)

print(f"승리 확률: {p_coin}, 배당률: {b_coin}")
print(f"최적의 베팅 비율(켈리 비율): {kelly_fraction:.2%}") 

승리 확률: 0.3, 배당률: 3
최적의 베팅 비율(켈리 비율): 6.67%


In [5]:
def calculate_asymmetric_kelly(p: float, win_rate: float, loss_rate: float) -> float:
    """
    비대칭 손익률을 위한 켈리 방정식을 계산합니다.

    Args:
        p (float): 승리 확률 (0과 1 사이)
        win_rate (float): 승리 시 베팅액 대비 수익률 (예: 20% 수익 시 0.2)
        loss_rate (float): 패배 시 베팅액 대비 손실률 (예: 10% 손실 시 0.1)

    Returns:
        float: 최적의 베팅 비율 f*. 0보다 작으면 0을 반환.
    """
    if not (0 <= p <= 1):
        raise ValueError("승리 확률(p)은 0과 1 사이의 값이어야 합니다.")
    if win_rate <= 0 or loss_rate <= 0:
        raise ValueError("수익률(win_rate)과 손실률(loss_rate)은 0보다 커야 합니다.")

    q = 1 - p

    # 기대 수익률이 0보다 작거나 같으면 베팅하지 않음
    # (p * win_rate) > (q * loss_rate) 이어야 베팅의 가치가 있음
    if (p * win_rate) <= (q * loss_rate):
        return 0.0

    # 비대칭 손익 켈리 공식
    # f* = (p*w - q*l) / (w*l)
    numerator = p * win_rate - q * loss_rate
    denominator = win_rate * loss_rate
    
    f_star = numerator / denominator
    
    return max(0, f_star)

# 예시: 승률 60%, 이기면 20% 얻고, 지면 10% 잃는 경우
p_custom = 0.6
w_custom = 0.2  # 20% 수익
l_custom = 0.1  # 10% 손실

optimal_fraction = calculate_asymmetric_kelly(p_custom, w_custom, l_custom)

print(f"승률: {p_custom:.0%}, 수익률: {w_custom:.0%}, 손실률: {l_custom:.0%}")
print(f"최적의 베팅 비율(켈리 비율): {optimal_fraction:.2%}")
# 만약 f* = 4.0 이라면 400.00% 로 출력됩니다.

승률: 60%, 수익률: 20%, 손실률: 10%
최적의 베팅 비율(켈리 비율): 400.00%


In [6]:
calculate_asymmetric_kelly(0.6, 0.2, 0.1)

3.9999999999999987

## 연습문제
1. 어떤 특별한 동전은 앞면이 나올 확률이 55%라고 합니다. 당신은 친구와 이 동전으로 내기를 합니다. 돈을 걸고 맞추면 건 돈의 1배를 받고, 틀리면 건 돈을 모두 잃습니다. 이 내기에 참여한다면, 자본의 몇 %를 베팅해야 할까요?

In [9]:
calculate_kelly_fraction(0.55, 1)

0.10000000000000009

2. 어떤 경주마가 우승할 확률이 40%라고 분석했습니다. 이 말에게 베팅하여 우승하면, 베팅액의 2.5배를 순수익으로 얻습니다. 이 경주마에게 베팅한다면 최적의 베팅 비율은 얼마일까요?

In [10]:
calculate_kelly_fraction(0.4, 2.5)

0.16

3. 룰렛 게임에 참여하려고 합니다. 특정 숫자에 베팅하여 이길 확률은 30%이고, 이기면 베팅액의 2배를 순수익으로 얻습니다. 이 게임에 베팅하는 것이 합리적일까요? 만약 합리적이지 않다면 그 이유는 무엇이며, 최적의 베팅 비율은 얼마일까요?

In [11]:
calculate_kelly_fraction(0.3, 2)

# 투자하면 안된다

0

4. 어떤 단타 매매 전략의 승률이 50%로 알려져 있습니다. 이 전략은 이기면 투자금의 15%를 벌고, 져도 투자금의 15%만 손절하여 손실을 제한합니다. 이 전략의 최적 포지션 사이즈(자본 대비 투자 비율)는 얼마일까요?

In [14]:
calculate_asymmetric_kelly(0.5, 0.15, 0.15)

# 투자하면 안된다

0.0

5. 당신은 익절은 길게, 손절은 짧게 하는 매매 전략을 만들었습니다. 분석 결과, 이 전략의 승률은 45%에 불과하지만, 이기면 투자금의 30% 수익을 내고, 지면 투자금의 10%만 잃습니다. 이 전략에 대한 최적의 포지션 사이즈는 얼마일까요?

In [12]:
calculate_asymmetric_kelly(0.45, 0.3, 0.1)

# 레버리지 포함 2.6%면 안하는게좋다

2.666666666666667

6. 승률이 70%로 매우 높은 매매 전략이 있습니다. 하지만 이기면 투자금의 10%를 버는 반면, 한 번 지면 투자금의 30%를 잃습니다. 이 전략은 사용할 가치가 있을까요? 최적 포지션 사이즈는 얼마일까요?

In [13]:
calculate_asymmetric_kelly(0.7, 0.1, 0.3)

# 투자하면 안된다

0.0